<a href="https://colab.research.google.com/github/Sahilk5/ASB/blob/main/ABL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("Welcome to my Programming Language")

Welcome to my Programming Language


In [3]:
import keyword
import re


In [4]:
# Environment name storage

class Environment:
  # record - Store of all local variables
  def __init__(self, record = {}, parent = None):
    self.record = record
    self.parent = parent
  
  # Identifier resolution
  def resolve(self, name):
    if name in self.record:
      return self
    
    if self.parent is None:
      raise NameError("No variable named '{}' found".format(name))
    
    return self.parent.resolve(name)

  # Add name entry in record
  def define(self, name, value):
    self.record[name] = value
    return value
  
  # Update variable if present in record
  def assign(self, name, value):
    self.resolve(name).record[name] = value
    return value

  # Find variable in current env and parent enviroments
  def lookupVar(self, name):
    return self.resolve(name).record[name]


In [7]:
class Abl:
    # Create an Abl instance with global env
    def __init__(self, Global = Environment()):
      self.Global = Global

    # Evaluates all the expressions withing a block
    def __eval_block(self, block, env):
      result = 0
      block = block[1:]
      for exp in block:
        result = self.evaluate(exp, env)
      return result

    # Evaluate an expression in the given env
    def evaluate(self, exp, env = None):
      # Set env if needed
      if not env: env = self.Global
      # Data types
      if isNumber(exp):
        return exp
      
      if isString(exp):
        return exp[1:-1]
      # Math operations
      if exp[0] is '+':
        return self.evaluate(exp[1], env) + self.evaluate(exp[2], env)
      
      if exp[0] is '-':
        return self.evaluate(exp[1], env) - self.evaluate(exp[2], env)

      if exp[0] is '*':
        return self.evaluate(exp[1], env) * self.evaluate(exp[2], env)

      if exp[0] is '/':
        return self.evaluate(exp[1], env) / self.evaluate(exp[2], env)
      
      # Comparison operators
      if exp[0] is '>':
        return self.evaluate(exp[1], env) > self.evaluate(exp[2], env)
      
      if exp[0] is '<':
        return self.evaluate(exp[1], env) < self.evaluate(exp[2], env)

      if exp[0] is '==':
        return self.evaluate(exp[1], env) == self.evaluate(exp[2], env)

      if exp[0] is '>=':
        return self.evaluate(exp[1], env) >= self.evaluate(exp[2], env)
      
      if exp[0] is '<=':
        return self.evaluate(exp[1], env) <= self.evaluate(exp[2], env)
      # Block
      if exp[0] is 'begin':
        block_env = Environment({}, env)
        return self.__eval_block(exp, block_env)

      # Variables
      # Declare
      if exp[0] is 'var':
        [_, name, value] = exp
        return env.define(name, self.evaluate(value, env))
      # Set
      if exp[0] is 'set':
        [_, name, value] = exp
        return env.assign(name, self.evaluate(value, env))
      # Access
      if isVariableName(exp):
        if len(exp) is 1:
          exp = exp[0]
        return env.lookupVar(exp)

      # if expression
      if(exp[0] is 'if'):
        [_tag, condition, consequent, alternate] = exp
        if self.evaluate(condition):
          return self.evaluate(consequent, env)
        else:
          return self.evaluate(alternate, env)

      # Exception
      raise Exception ("Unimplemented  {}".format(exp))




def isNumber(exp):
  if type(exp) is int or type(exp) is float:
    return True
  return False

def isString(exp):
  return type(exp) is str and exp[0] is '"' and exp[-1] is '"'

def isVariableName(exp):
  if len(exp) is 1:
    exp = exp[0]
  variableNameFormat = re.compile("^([a-zA-Z])[a-zA-Z0-9_]*$")
  return type(exp) is str and variableNameFormat.match(exp)

In [9]:
# Tests

abl = Abl(Environment(
    {
        'True'  : True,
        'False' : False,
        'None'  : None,
        'VERSION' : '1.0'
    }
))

# Math
assert abl.evaluate(1) == 1
assert abl.evaluate(1.0) == 1.0
assert abl.evaluate('"Hello World"') == 'Hello World'

assert abl.evaluate(['+', 1, 3]) == 4
assert abl.evaluate(['+', 1.2, 3.3]) == 4.5
assert abl.evaluate(['+', ['+', 1, 4], 4]) == 9
assert abl.evaluate(['-', 4, 3]) == 1
assert abl.evaluate(['*', 6, 3]) == 18
assert abl.evaluate(['/', 9, 3]) == 3

assert abl.evaluate(['+', '"Hello "', '"World"']) == "Hello World"

# Var
assert abl.evaluate(['var', 'x', 10]) == 10
assert abl.evaluate(['x']) == 10
assert abl.evaluate(['var', 'y', 20]) == 20
assert abl.evaluate(['var', 'flag', 'True']) == True
assert abl.evaluate(['var', 'xy', ['*', 'x', 'y']]) == 200
assert abl.evaluate(['VERSION']) == '1.0'

# Blocks
assert abl.evaluate(
    ['begin',
     ['var', 'x', 10],
     ['var', 'y', 20],
     ['+', ['*', 'x', 'y'], 30],
     ])  == 230

assert abl.evaluate(
    ['begin',
     ['var', 'x', 10],
     ['begin',
      ['var', 'x', 20],
      ['x']],
     'x']) == 10

assert abl.evaluate(
    ['begin',
     ['var', 'x', 10],
     ['begin',
      ['set', 'x', 20],
      ['x']],
     'x']) == 20

assert abl.evaluate(
    ['begin',
     ['var', 'a', 10],
     ['var', 'b', ['begin',
                   ['var', 'x', ['+', 'a', 10]],
                   ['x']]],
     ['b']]) == 20

# if condition
assert abl.evaluate(
    ['begin',
     ['var', 'x', 10],
     ['var', 'y', 0],
     ['if', ['>', 'x', 10],
      ['set', 'y', 20],
      ['set', 'y', 30],],
     ['y']]) == 30


print("All assertions succeded")

All assertions succeded
